# Big G Express Data Preparaton & Preliminary Explorations

## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Read in

In [2]:
faults = pd.read_csv('../data/J1939Faults.csv')
diag = pd.read_csv('../data/VehicleDiagnosticOnboardData.csv')
makes = pd.read_excel('../data/Vehicle_Make.xlsx')
fault_codes = pd.read_excel('../data/Service Fault Codes_1_0_0_167.xlsx')


C:\Users\jrior\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\jrior\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


## Glance at each df, convert datatypes, and prepare for merging

### faults
- 20 columns
- 1,187,335 rows
- EventTimeStamp, LocationTimeStamp cast as datetimes
- Many spn-fmi combinations have multiple rows in the dataset, usually one with an Algorithmic Description and another without one
- 1,187,335 distinct FaultIds

In [3]:
faults.shape

(1187335, 20)

In [4]:
faults.head()

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000


In [5]:
for col in ['EventTimeStamp', 'LocationTimeStamp']:
    faults[col] = faults[col].astype('datetime64[ns]')

### diag (from Onboard Diagnostic Data)
- 4 columns
- 12,821,626 rows
- The data is in long form. The numbers use commas as decimals, so we'll replace those so we can change the wideform columns to floats.

In [7]:
diag.shape

(12821626, 4)

In [9]:
diag.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12821626 entries, 0 to 12821625
Data columns (total 4 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   Id       int64 
 1   Name     object
 2   Value    object
 3   FaultId  int64 
dtypes: int64(2), object(2)
memory usage: 391.3+ MB


In [10]:
diag['FaultId'].nunique()

1187335

In [11]:
diag.loc[diag['Value'].str.contains(',')]

,Id,Name,Value,FaultId
200754,201459,EngineOilPressure,"75,98",18402
200755,201460,EngineOilTemperature,"177,575",18402
200757,201462,TurboBoostPressure,"33,35",18402
200760,201465,FuelLtd,"17867,93716715",18402
200761,201466,FuelRate,"20,68474",18402
...,...,...,...,...
5632164,5640222,IntakeManifoldTemperature,"98,6",545727
5632166,5640224,EngineRpm,"1261,5",545727
5632167,5640225,BarometricPressure,"14,2825",545727
5632169,5640227,Speed,"64,47697",545727


In [12]:
diag.loc[diag['Value'].str.contains(','), 'Name'].unique()

array(['EngineOilPressure', 'EngineOilTemperature', 'TurboBoostPressure',
       'FuelLtd', 'FuelRate', 'IntakeManifoldTemperature', 'EngineRpm',
       'BarometricPressure', 'FuelLevel', 'Speed', 'EngineTimeLtd',
       'EngineCoolantTemperature', 'SwitchedBatteryVoltage',
       'DistanceLtd', 'FuelTemperature', 'Throttle', 'AcceleratorPedal'],
      dtype=object)

In [13]:
diag['Value'] = diag['Value'].str.replace(',', '.')

## fault_codes

- Collection of references to interpret the faults spn and fmi columns
- Focus here is therefore on SPN, J1939 FMI (FMI pertaining to derates), as well as the interpretations: Lamp Color, Lamp Device, Cummins Description, Algorithm Description
- Some SPN, J1939 FMI pairs have multiple rows, and in some cases the differences are nontrivial. For example, these lines:

    |Cummins Fault Code|	SPN	|J1939 FMI	|Lamp Color|	Lamp Device|	Cummins Description	|Algorithm Description|
    |------------------|---------|----------|----------|----------------|-----------------------|----------------------|
    |2519|	51|	2|	Red|	Stop / Shutdown|	Throttle Position Sensor - Data Erratic, Inter...	|Throttle actuator control lost|
    |3542|	51|	2|	Amber|	Warning	|Engine Intake Throttle Actuator Position Senso...	|NaN|

In [14]:
fault_codes.shape

(7124, 14)

In [52]:
fault_codes.sort_values(['SPN', 'J1939 FMI']).head(6)

,Cummins Fault Code,SPN,J1939 FMI,Lamp Color,Lamp Device,Cummins Description,Algorithm Description
470,719,22,3,Amber,Warning,Extended Crankcase Blow-by Pressure Circuit - ...,High voltage has been detected in the Extended...
479,729,22,4,Amber,Warning,Extended Crankcase Blow-by Pressure Circuit - ...,Low voltage has been detected in the Extended ...
734,1228,27,2,Amber,Warning,"EGR Valve Position - Data Erratic, Intermitten...",NaN
1457,2271,27,3,Amber,Warning,EGR Valve Position Circuit - Voltage Above Nor...,High signal voltage detected at EGR Valve Posi...
1458,2272,27,4,Amber,Warning,EGR Valve Position Circuit - Voltage Below Nor...,Low signal voltage has been detected at the EG...
1515,2348,27,13,Amber,Warning,EGR Valve Position Circuit - Out of Calibration,The EGR valve position is out of calibration.


In [18]:
fault_codes = fault_codes[['Cummins Fault Code', 'SPN', 'J1939 FMI', 'Lamp Color', 'Lamp Device', 'Cummins Description', 'Algorithm Description']]

### makes

We've discovered that makes has no referential integrity with the rest of the data.

## Merge faults and diag

- Widen diag so that each fault name has its own column (though we'll get a lot of NaNs). Call this diag_wide.
- Make booleans actual boolean values rather than objects. Note that columns containing booleans and NaNs are technically mixed type and thus still "object" type.
- Cast the rest of the columns as floats.
- Merge faults and diag_wide on RecordID and FaultId, respectively. Called fdwide.
- Merge faults_diag_wide with fault_codes on spn and fmi.
    - This creates duplicate RecordID/FaultId values/rows because fault_codes has multiple lines for the same spn and fmi pairs. We should deal with this.
        - [ ] Option 1: Only take the first value, prioritizing those with non-null Algorithm Descriptions
        - [ ] Option 2: Widen the spn/fmi pairs to have multiple entries for Lamp Color, Lamp Device, Cummins Description, and Algorithm Description
        - [x] Option 3: Wait to incorporate fault_codes until we need to interpret results. Just work with fdwide.
- Pickle faults, diag, and fdwide for use in other notebooks.

In [54]:
diag_wide = diag.pivot(index = ['FaultId'],columns = 'Name', values = 'Value')

diag_wide_booleans = ['CruiseControlActive', 'IgnStatus', 'ParkingBrake']
boolean_mapper = {'True':True, 'False':False}

for col in diag_wide_booleans:
    diag_wide[col] = diag_wide[col].map(boolean_mapper)

for col in diag_wide.loc[:,~diag_wide.columns.isin(diag_wide_booleans)].columns:
    diag_wide[col] = diag_wide[col].astype(float)

fdwide = faults.merge(diag_wide.reset_index(), left_on = 'RecordID', right_on='FaultId')

full = fdwide.merge(fault_codes, 
                            left_on = ['spn', 'fmi'], 
                            right_on = ['SPN', 'J1939 FMI'], 
                            how = 'left'
    )

In [56]:
print('fdwide shape: ', fdwide.shape)
print('fdwide dedup shape: ', fdwide.drop_duplicates(keep = 'first').shape)
print('full: ', full.shape)
print('full dedup: ', full.drop_duplicates(subset = fdwide.columns, keep = 'first').shape)
print('full origin columns dedup: ', full[fdwide.columns].drop_duplicates(subset = fdwide.columns, keep = 'first').shape)
#full[~((full.duplicated(subset = ['RecordID', 'spn', 'fmi'], keep = False))&(full['Algorithm Description'].isnull()))]

fdwide shape:  (1187335, 45)
fdwide dedup shape:  (1187335, 45)
full:  (1773397, 52)
full dedup:  (1187335, 52)
full origin columns dedup:  (1187335, 45)


In [58]:
faults.to_pickle('../data/faults_df.pickle')
diag.to_pickle('../data/diag_df.pickle')
fdwide.to_pickle('../data/fdwide_df.pickle')